In [ ]:
%run -i '/content/drive/MyDrive/DeepLearning/util_simple_classifier.ipynb'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!git clone https://github.com/PacktPublishing/Python-Natural-Language-Processing-Cookbook-Second-Edition.git

Cloning into 'Python-Natural-Language-Processing-Cookbook-Second-Edition'...
remote: Enumerating objects: 433, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 433 (delta 11), reused 6 (delta 2), pack-reused 409 (from 1)
Receiving objects: 100% (433/433), 18.28 MiB | 18.21 MiB/s, done.
Resolving deltas: 100% (235/235), done.


In [ ]:
%cd Python-Natural-Language-Processing-Cookbook-Second-Edition

/content/Python-Natural-Language-Processing-Cookbook-Second-Edition


In [ ]:
import pandas as pd
from spacy.cli.train import train
from spacy.cli.evaluate import evaluate
from spacy.cli.debug_data import debug_data
from spacy.tokens import DocBin
from sklearn.metrics import classification_report
# Config generated at https://spacy.io/usage/training
import spacy
small_model = spacy.load("en_core_web_sm")
def preprocess_data_entry(input_text, label, label_list):
    doc = small_model(input_text)
    cats = [0] * len(label_list)
    cats[label] = 1
    final_cats = {}
    for i, label in enumerate(label_list):
        final_cats[label] = cats[i]
    doc.cats = final_cats
    return doc
# Load and prepare data
train_db = DocBin()
test_db = DocBin()
label_list = ["tech", "business", "sport", "entertainment", "politics"]
train_df = pd.read_json("/content/Python-Natural-Language-Processing-Cookbook-Second-Edition/data/bbc_train.json")
test_df = pd.read_json("/content/Python-Natural-Language-Processing-Cookbook-Second-Edition/data/bbc_test.json")
train_df.sample(frac=1)
for idx, row in train_df.iterrows():
    text = row["text"]
    label = row["label"]
    doc = preprocess_data_entry(text, label, label_list)
    train_db.add(doc)
for idx, row in test_df.iterrows():
    text = row["text"]
    label = row["label"]
    doc = preprocess_data_entry(text, label, label_list)
    test_db.add(doc)


FileNotFoundError: [Errno 2] No such file or directory: '../data/bbc_train.spacy'

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
train_db.to_disk('/content/Python-Natural-Language-Processing-Cookbook-Second-Edition/data/bbc_train.spacy')
test_db.to_disk('/content/Python-Natural-Language-Processing-Cookbook-Second-Edition/data/bbc_test.spacy')

!mv /content/Python-Natural-Language-Processing-Cookbook-Second-Edition/data/bbc_train.spacy /content/data/
!mv /content/Python-Natural-Language-Processing-Cookbook-Second-Edition/data/bbc_test.spacy /content/data/
# Now we can call train
train("/content/Python-Natural-Language-Processing-Cookbook-Second-Edition/data/spacy_config.cfg", output_path="/content/data")

ℹ Saving to output directory: /content/data
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ------------  ----------  ------


In [ ]:
# Use the trained model
nlp = spacy.load("../models/spacy_textcat_bbc/model-last")
input_text = test_df.iloc[1, test_df.columns.get_loc('text')]
print(input_text)
print(test_df["label_text"].iloc[[1]])
doc = nlp(input_text)
print("Predicted probabilities: ", doc.cats)

In [ ]:
# Evaluate the model on test data
def get_prediction(input_text, nlp_model, target_names):
    doc = nlp_model(input_text)
    category = max(doc.cats, key = doc.cats.get)
    return target_names.index(category)
test_df["prediction"] = test_df["text"].apply(lambda x: get_prediction(x, nlp, label_list))
print(classification_report(test_df["label"], test_df["prediction"], target_names=label_list))

In [ ]:
evaluate('../models/spacy_textcat_bbc/model-last', '../data/bbc_test.spacy')